In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List

In [2]:
class Layer:

    def __init__(self):
        self.training = True

    def forward(self, x):
        pass

    def backward(self, grad_output):
        pass

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    def __call__(self, x):
        return self.forward(x)

In [3]:
class RBF(Layer):

    def __init__(self, input_size, output_size, bias=True):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.use_bias = bias

        self.weight = np.random.normal(loc=0, scale=2/input_size, size=(input_size, output_size)).astype(np.float32)

        if self.use_bias:
            self.bias = np.zeros(output_size, dtype=np.float32)
        else:
            self.bias = None

    def forward(self, x):
        self.input = x

        output = np.square(self.input - self.weight)
        if self.use_bias:
            output = output - np.square(self.bias)
        return output

    def backward(self, grad_output):
        grad_input = grad_output.T @ (2 * self.input - 2 * self.weight).T

        self.grad_weight = grad_output @ (-2 * self.input + 2 * self.weight).T

        if self.use_bias:
            self.grad_bias = np.sum(grad_output, axis=0) * (2 * self.bias).T

        return grad_input

    def update_weights(self, learning_rate=0.01):
        if self.grad_weight is not None:
            self.weight.T -= learning_rate * self.grad_weight

        if self.use_bias and self.grad_bias is not None:
            self.bias -= learning_rate * self.grad_bias

In [4]:
### Функция активации ReLU

class ReLU(Layer):

    def __init__(self):
        super().__init__()
        self.input = None

    def forward(self, x):
        self.input = x
        output = np.maximum(0, x)
        return output

    def backward(self, grad_output):
        grad_input = grad_output.copy()
        grad_input[self.input <= 0] = 0 ### При отрицательных значениях входных данных ReLU превращается в горизонтальную прямую, её производная ноль
                                   ### При значениях больше нуля представляет собой прямую с коэффициентом угла наклона 1
                                   ### Если я правильно помню в чем смысл backward у функции активации, то мы возвращаем произведение весов на grad
                                   ### То есть когда производная равна 1, мы возвращаем просто веса
        return grad_input

In [5]:
# Тест ReLU (запустите этот код после реализации ReLU)
relu = ReLU()

# # Тестовые данные
x_test = np.array([[-2, -1, 0, 1, 2]], dtype=np.float32)
expected_forward = np.array([[0, 0, 0, 1, 2]], dtype=np.float32)

# # Forward pass
output = relu.forward(x_test)
print(f"Input: {x_test}")
print(f"Output: {output}")
print(f"Expected: {expected_forward}")

# # Проверка forward pass
assert np.allclose(output, expected_forward), "ReLU forward pass не работает корректно!"

# # Backward pass
grad_output = np.ones_like(output)
grad_input = relu.backward(grad_output)
expected_backward = np.array([[0, 0, 0, 1, 1]], dtype=np.float32)

print(f"Gradient output: {grad_output}")
print(f"Gradient input: {grad_input}")
print(f"Expected gradient: {expected_backward}")

# # Проверка backward pass
assert np.allclose(grad_input, expected_backward), "ReLU backward pass не работает корректно!"

print("✅ ReLU тест пройден успешно!")

#print("⚠️ Реализуйте ReLU класс выше, затем раскомментируйте этот код для тестирования")


Input: [[-2. -1.  0.  1.  2.]]
Output: [[0. 0. 0. 1. 2.]]
Expected: [[0. 0. 0. 1. 2.]]
Gradient output: [[1. 1. 1. 1. 1.]]
Gradient input: [[0. 0. 0. 1. 1.]]
Expected gradient: [[0. 0. 0. 1. 1.]]
✅ ReLU тест пройден успешно!


In [6]:
### Функция активации Sigmoid

class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
        self.output = None

    def forward(self, x):
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, grad_output):
        ### Из чистого интереса брал производную, выходит (е^(-x)/(1+е^(-x))^2)
        ### Так понимаю именно её мы не используем в силу того, что появляется дополнительная вычислительная сложность
        return grad_output * self.output * (1 - self.output)

In [7]:
# Тест Sigmoid (запустите этот код после реализации Sigmoid)
#print("⚠️ Реализуйте Sigmoid класс выше, затем раскомментируйте этот код для тестирования")

sigmoid = Sigmoid()

# # Тестовые данные
x_test = np.array([[-10, -1, 0, 1, 10]], dtype=np.float32)

# # Forward pass
output = sigmoid.forward(x_test)
print(f"Input: {x_test}")
print(f"Output: {output}")

# # Проверим, что выходные значения в диапазоне (0, 1)
assert np.all(output > 0) and np.all(output < 1), "Sigmoid должен возвращать значения в диапазоне (0, 1)"

# # Проверим симметричность: sigmoid(-x) = 1 - sigmoid(x)
x_sym = np.array([[1]], dtype=np.float32)
out_pos = sigmoid.forward(x_sym)
out_neg = sigmoid.forward(-x_sym)
assert np.allclose(out_neg, 1 - out_pos, atol=1e-6), "Sigmoid должен быть симметричным"

# # Backward pass
grad_output = np.ones_like(output)
grad_input = sigmoid.backward(grad_output)
print(f"Gradient input: {grad_input}")

# # Проверим, что градиент положительный (sigmoid монотонно возрастает)
assert np.all(grad_input >= 0), "Градиент Sigmoid должен быть неотрицательным"

print("✅ Sigmoid тест пройден успешно!")


Input: [[-10.  -1.   0.   1.  10.]]
Output: [[4.5397872e-05 2.6894143e-01 5.0000000e-01 7.3105860e-01 9.9995458e-01]]
Gradient input: [[0.19661194 0.19661194 0.19661194 0.19661194 0.19661194]]
✅ Sigmoid тест пройден успешно!


In [8]:
### Функция активации Tanh

class Tanh(Layer):
    def __init__(self):
        super().__init__()
        self.output = None

    def forward(self, x):
        self.output = np.clip((np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x)), -0.9999, 0.9999)
        return self.output

    def backward(self, grad_output):
        return grad_output * (1 - np.square(self.output))

In [9]:
# Тест Tanh (запустите этот код после реализации Tanh)
print("⚠️ Реализуйте Tanh класс выше, затем раскомментируйте этот код для тестирования")

tanh = Tanh()

# # Тестовые данные
x_test = np.array([[-10, -1, 0, 1, 10]], dtype=np.float32)

# # Forward pass
output = tanh.forward(x_test)
print(f"Input: {x_test}")
print(f"Output: {output}")

# # Проверим, что выходные значения в диапазоне (-1, 1)
assert np.all(output > -1) and np.all(output < 1), "Tanh должен возвращать значения в диапазоне (-1, 1)"

# # Проверим антисимметричность: tanh(-x) = -tanh(x)
x_antisym = np.array([[2]], dtype=np.float32)
out_pos = tanh.forward(x_antisym)
out_neg = tanh.forward(-x_antisym)
assert np.allclose(out_neg, -out_pos, atol=1e-6), "Tanh должен быть антисимметричным"

# # Проверим, что tanh(0) = 0
zero_out = tanh.forward(np.array([[0]], dtype=np.float32))
assert np.allclose(zero_out, 0, atol=1e-6), "tanh(0) должен быть равен 0"

# # Backward pass
grad_output = np.ones_like(output)
grad_input = tanh.backward(grad_output)
print(f"Gradient input: {grad_input}")

# # Проверим, что градиент положительный (tanh монотонно возрастает)
assert np.all(grad_input >= 0), "Градиент Tanh должен быть неотрицательным"

print("✅ Tanh тест пройден успешно!")


⚠️ Реализуйте Tanh класс выше, затем раскомментируйте этот код для тестирования
Input: [[-10.  -1.   0.   1.  10.]]
Output: [[-0.9999    -0.7615942  0.         0.7615942  0.9999   ]]
Gradient input: [[1. 1. 1. 1. 1.]]
✅ Tanh тест пройден успешно!


In [10]:
### Линейный слой

class Linear(Layer):
    def __init__(self, input_size, output_size, bias=True):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.use_bias = bias

        ### Kaiming He инициализация весов
        self.weight = np.random.normal(loc=0, scale=2/input_size, size=(input_size, output_size)).astype(np.float32)

        if self.use_bias:
            self.bias = np.zeros(output_size, dtype=np.float32)
        else:
            self.bias = None

    def forward(self, x):
        self.input = x

        output = self.weight.T @ x.T
        if self.use_bias:
            output = output.T + self.bias
        return output

    def backward(self, grad_output):
        grad_input = grad_output @ self.weight.T

        self.grad_weight = self.input.T @ grad_output

        if self.use_bias:
            self.grad_bias = np.sum(grad_output, axis=0)

        return grad_input

    def update_weights(self, learning_rate=0.01):
        if self.grad_weight is not None:
            self.weight.T -= learning_rate * self.grad_weight

        if self.use_bias and self.grad_bias is not None:
            self.bias -= learning_rate * self.grad_bias

In [11]:
# Тест Linear (запустите этот код после реализации Linear)
#print("⚠️ Реализуйте Linear класс выше, затем раскомментируйте этот код для тестирования")

linear = Linear(input_size=3, output_size=2, bias=True)

# # Проверим форму весов
assert linear.weight.shape == (3, 2), f"Неправильная форма весов: {linear.weight.shape}"
assert linear.bias.shape == (2,), f"Неправильная форма bias: {linear.bias.shape}"

print(f"Веса: \n{linear.weight}")
print(f"Bias: {linear.bias}")

# # Тестовые данные
batch_size = 4
x_test = np.random.randn(batch_size, 3).astype(np.float32)

# # Forward pass
output = linear.forward(x_test)
expected_shape = (batch_size, 2)

print(f"Input shape: {x_test.shape}")
print(f"Output shape: {output.shape}")
print(f"Expected shape: {expected_shape}")

assert output.shape == expected_shape, f"Неправильная форма выхода: {output.shape}"

# # Backward pass
grad_output = np.random.randn(*output.shape).astype(np.float32)
grad_input = linear.backward(grad_output)

print(f"Gradient input shape: {grad_input.shape}")
print(f"Gradient weight shape: {linear.grad_weight.shape}")
print(f"Gradient bias shape: {linear.grad_bias.shape}")

# # Проверим формы градиентов
assert grad_input.shape == x_test.shape, "Неправильная форма градиента по входу"
assert linear.grad_weight.shape == linear.weight.shape, "Неправильная форма градиента по весам"
assert linear.grad_bias.shape == linear.bias.shape, "Неправильная форма градиента по bias"

print("✅ Linear тест пройден успешно!")

Веса: 
[[-0.08106621 -1.4308075 ]
 [-0.1198379  -1.1728611 ]
 [ 0.21903415  1.1588073 ]]
Bias: [0. 0.]
Input shape: (4, 3)
Output shape: (4, 2)
Expected shape: (4, 2)
Gradient input shape: (4, 3)
Gradient weight shape: (3, 2)
Gradient bias shape: (2,)
✅ Linear тест пройден успешно!


In [12]:
### Последовательный контейнер (Sequential)

class Sequential(Layer):
    def __init__(self, *layers):
        super().__init__()
        self.layers = list(layers)
        self.layer_outputs = [] ### Не понял зачем это здесь нужно, ведь дальше нигде не используется

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, x):
        self.layer_outputs = []
        output = x
        for layer in self.layers:
            self.layer_outputs.append(output)
            output = layer.forward(output)

        return output

    def backward(self, grad_output):
        grad = grad_output
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        return grad

    def train(self):
        super().train()
        for layer in self.layers:
            layer.train()

    def eval(self):
        super().eval()
        for layer in self.layers:
            layer.eval()

    def __len__(self):
        return len(self.layers)

    def __getitem__(self, idx):
        return self.layers[idx]

In [13]:
### Dropout

class Dropout(Layer):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.mask = None

    def forward(self, x):
        if self.training:
            self.mask = np.random.uniform(0, 1, size=np.shape(x))
            x[self.mask > self.dropout_rate] = 0
            output = x
        else:
            output = x
            self.mask = None
        return output

    def backward(self, grad_output):
        if self.training:
            grad_output[self.mask > self.dropout_rate] = 0
            grad_input = grad_output
        else:
            grad_input = grad_output
        return grad_input

In [14]:
# Тест Dropout (запустите после реализации Dropout)
#print("⚠️ Реализуйте Dropout класс выше, затем раскомментируйте этот код для тестирования")

dropout = Dropout(dropout_rate=0.5)

# # Тестовые данные
x_test = np.ones((100, 10), dtype=np.float32)

# # Тест в режиме обучения
dropout.train()
output_train = dropout.forward(x_test)

print(f"Режим обучения:")
print(f"Input mean: {x_test.mean():.3f}")
print(f"Output mean: {output_train.mean():.3f}")
print(f"Proportion of zeros: {(output_train == 0).mean():.3f}")

# # Проверим, что часть нейронов "выключена"
zeros_ratio = (output_train == 0).mean()
expected_zeros = 0.5  # dropout_rate
assert abs(zeros_ratio - expected_zeros) < 0.1, f"Неправильная доля нулевых значений: {zeros_ratio}"

# # Проверим масштабирование
assert abs(output_train.mean() - x_test.mean()) < 0.1, "Неправильное масштабирование в режиме обучения"

# # Тест в режиме инференса
dropout.eval()
output_eval = dropout.forward(x_test)

print(f"\nРежим инференса:")
print(f"Output mean: {output_eval.mean():.3f}")
print(f"Proportion of zeros: {(output_eval == 0).mean():.3f}")

# # В режиме инференса все значения должны остаться
assert np.allclose(output_eval, x_test), "В режиме инференса выход должен совпадать с входом"

# # Тест backward pass
dropout.train()
output_train = dropout.forward(x_test)
grad_output = np.ones_like(output_train)
grad_input = dropout.backward(grad_output)

print(f"\nGradient test:")
print(f"Grad input shape: {grad_input.shape}")
print(f"Grad input mean: {grad_input.mean():.3f}")

assert grad_input.shape == x_test.shape, "Неправильная форма градиента"

print("✅ Dropout тест пройден успешно!")

Режим обучения:
Input mean: 0.521
Output mean: 0.521
Proportion of zeros: 0.479

Режим инференса:
Output mean: 0.521
Proportion of zeros: 0.479

Gradient test:
Grad input shape: (100, 10)
Grad input mean: 0.501
✅ Dropout тест пройден успешно!


In [15]:
### Batch Normalization

class BatchNorm(Layer):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum

        self.gamma = np.ones(num_features, dtype=np.float32)
        self.beta = np.zeros(num_features, dtype=np.float32)

        self.running_mean = np.zeros(num_features, dtype=np.float32)
        self.running_var = np.ones(num_features, dtype=np.float32)

        self.batch_mean = None
        self.batch_var = None
        self.normalized = None
        self.input = None
        self.grad_gamma = None
        self.grad_beta = None

    def forward(self, x):
        self.input = x.astype(np.float32)
        #self.input = x

        if self.training:
            self.batch_mean = np.mean(x, axis=0, keepdims=True).astype(np.float32)
            self.batch_var = np.var(x, axis=0, keepdims=True).astype(np.float32)

            self.running_mean = (self.momentum * self.batch_mean + (1 - self.momentum) * self.running_mean).astype(np.float32)
            self.running_var = (self.momentum * self.batch_var + (1 - self.momentum) * self.running_var).astype(np.float32)

            mean = self.batch_mean
            var = self.batch_var
        else:
            mean = self.running_mean.reshape(1, -1)
            var = self.running_var.reshape(1, -1)

        self.normalized = ((x - mean) / np.sqrt(var + self.eps)).astype(np.float32)

        output = (self.gamma * self.normalized + self.beta).astype(np.float32)

        return output

    def backward(self, grad_output):
        batch_size = grad_output.shape[0]
        grad_output = grad_output.astype(np.float32)

        self.grad_gamma = np.sum(grad_output * self.normalized, axis=0).astype(np.float32)
        self.grad_beta = np.sum(grad_output, axis=0).astype(np.float32)
        ### И тут начался сущий кошмар
        ### Честно признаюсь, списал, но даже так производная BatchNorm это какая-то жесть
        ### Списал отсюда: https://blog.tnichols.org/posts/batchnorm-backward/

        if self.training:
            grad_normalized = grad_output * self.gamma
            grad_var = np.sum(grad_normalized * (self.input - self.batch_mean) * -0.5 * 
                            np.power(self.batch_var + self.eps, -1.5), axis=0, keepdims=True)
            grad_mean = np.sum(grad_normalized * -1.0 / np.sqrt(self.batch_var + self.eps), axis=0, keepdims=True) + \
                       grad_var * np.mean(-2.0 * (self.input - self.batch_mean), axis=0, keepdims=True)
            
            grad_input = (grad_normalized / np.sqrt(self.batch_var + self.eps) + 
                         grad_var * 2.0 * (self.input - self.batch_mean) / batch_size + 
                         grad_mean / batch_size).astype(np.float32)
        else:
            grad_input = (grad_output * self.gamma / np.sqrt(self.running_var.reshape(1, -1) + self.eps)).astype(np.float32)
        
        return grad_input

    def update_weights(self, learning_rate=0.001):
        """
        Обновление параметров gamma и beta
        """
        if self.grad_gamma is not None:
            self.gamma -= learning_rate * self.grad_gamma
        
        if self.grad_beta is not None:
            self.beta -= learning_rate * self.grad_beta

In [16]:
# Тест BatchNorm (запустите после реализации BatchNorm)
#print("⚠️ Реализуйте BatchNorm класс выше, затем раскомментируйте этот код для тестирования")

batch_norm = BatchNorm(num_features=4)

# # Проверим инициализацию
assert np.allclose(batch_norm.gamma, 1.0), "Gamma должно инициализироваться единицами"
assert np.allclose(batch_norm.beta, 0.0), "Beta должно инициализироваться нулями"
assert np.allclose(batch_norm.running_mean, 0.0), "Running mean должно инициализироваться нулями"
assert np.allclose(batch_norm.running_var, 1.0), "Running var должно инициализироваться единицами"

print(f"Gamma: {batch_norm.gamma}")
print(f"Beta: {batch_norm.beta}")

# # Тестовые данные с известной статистикой
x_test = np.array([
     [1, 2, 3, 4],
     [2, 3, 4, 5],
     [3, 4, 5, 6]
 ], dtype=np.float32)

print(f"Input: \n{x_test}")
print(f"Input mean per feature: {x_test.mean(axis=0)}")
print(f"Input std per feature: {x_test.std(axis=0)}")

# # Forward pass в режиме обучения
batch_norm.train()
output = batch_norm.forward(x_test)

print(f"\nOutput: \n{output}")
print(f"Output mean per feature: {output.mean(axis=0)}")
print(f"Output std per feature: {output.std(axis=0)}")

# # Проверим, что выход нормализован (среднее ≈ 0, std ≈ 1)
assert np.allclose(output.mean(axis=0), 0, atol=1e-6), "Среднее должно быть близко к 0"
assert np.allclose(output.std(axis=0), 1, atol=1e-6), "Стандартное отклонение должно быть близко к 1"

# # Проверим обновление running статистики
print(f"\nRunning mean: {batch_norm.running_mean}")
print(f"Running var: {batch_norm.running_var}")

# # Backward pass
grad_output = np.ones_like(output)
grad_input = batch_norm.backward(grad_output)

print(f"\nGradient input shape: {grad_input.shape}")
print(f"Gradient gamma shape: {batch_norm.grad_gamma.shape}")
print(f"Gradient beta shape: {batch_norm.grad_beta.shape}")

assert grad_input.shape == x_test.shape, "Неправильная форма градиента по входу"
assert batch_norm.grad_gamma.shape == batch_norm.gamma.shape, "Неправильная форма градиента gamma"
assert batch_norm.grad_beta.shape == batch_norm.beta.shape, "Неправильная форма градиента beta"

# # Тест режима инференса
batch_norm.eval()
output_eval = batch_norm.forward(x_test)
print(f"\nInference mode output mean: {output_eval.mean(axis=0)}")

print("✅ BatchNorm тест пройден успешно!")

Gamma: [1. 1. 1. 1.]
Beta: [0. 0. 0. 0.]
Input: 
[[1. 2. 3. 4.]
 [2. 3. 4. 5.]
 [3. 4. 5. 6.]]
Input mean per feature: [2. 3. 4. 5.]
Input std per feature: [0.8164966 0.8164966 0.8164966 0.8164966]

Output: 
[[-1.2247356 -1.2247356 -1.2247356 -1.2247356]
 [ 0.         0.         0.         0.       ]
 [ 1.2247356  1.2247356  1.2247356  1.2247356]]
Output mean per feature: [0. 0. 0. 0.]
Output std per feature: [0.99999243 0.99999243 0.99999243 0.99999243]

Running mean: [[0.2 0.3 0.4 0.5]]
Running var: [[0.96666664 0.96666664 0.96666664 0.96666664]]

Gradient input shape: (3, 4)
Gradient gamma shape: (4,)
Gradient beta shape: (4,)

Inference mode output mean: [1.8307619 2.746143  3.6615238 4.5769053]
✅ BatchNorm тест пройден успешно!


In [17]:
### Adam Оптимизатор

class Adam:

    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        self.learning_rate = np.float32(learning_rate)
        self.beta1 = np.float32(beta1)
        self.beta2 = np.float32(beta2)
        self.eps = np.float32(eps)
        
        # Словари для хранения моментов для каждого слоя
        self.m = {}  # first moment
        self.v = {}  # second moment
        self.t = 0   # time step

    def update(self, layer, layer_id):
        self.t += 1

        if hasattr(layer, 'grad_weight') and layer.grad_weight is not None:
            if f"{layer_id}_weight" not in self.m:
                self.m[f"{layer_id}_weight"] = np.zeros_like(layer.grad_weight, dtype=np.float32)
                self.v[f"{layer_id}_weight"] = np.zeros_like(layer.grad_weight, dtype=np.float32)
            # Обновление моментов
            self.m[f"{layer_id}_weight"] = (self.beta1 * self.m[f"{layer_id}_weight"] + (1 - self.beta1) * layer.grad_weight).astype(np.float32)
            self.v[f"{layer_id}_weight"] = (self.beta2 * self.v[f"{layer_id}_weight"] + (1 - self.beta2) * np.square(layer.grad_weight)).astype(np.float32)
            # Коррекция смещения
            m_corrected = self.m[f"{layer_id}_weight"] / (1 - self.beta1 ** self.t).astype(np.float32)
            v_corrected = self.v[f"{layer_id}_weight"] / (1 - self.beta2 ** self.t).astype(np.float32)
            #Обновление весов
            layer.weight -= self.learning_rate * m_corrected / (np.sqrt(v_corrected) + self.eps).astype(np.float32)

        if hasattr(layer, 'grad_bias') and layer.grad_bias is not None:
            if f"{layer_id}_bias" not in self.m:
                self.m[f"{layer_id}_bias"] = np.zeros_like(layer.grad_bias, dtype=np.float32)
                self.v[f"{layer_id}_bias"] = np.zeros_like(layer.grad_bias, dtype=np.float32)
            
            # TODO: Реализуйте обновление bias
            self.m[f"{layer_id}_bias"] = (self.beta1 * self.m[f"{layer_id}_bias"] + (1 - self.beta1) * layer.grad_bias).astype(np.float32)
            self.v[f"{layer_id}_bias"] = (self.beta2 * self.v[f"{layer_id}_bias"] + (1 - self.beta2) * np.square(layer.grad_bias)).astype(np.float32)
            
            m_corrected_bias = (self.m[f"{layer_id}_bias"] / ( 1 - self.beta1 ** self.t)).astype(np.float32)
            v_corrected_bias = (self.v[f"{layer_id}_bias"] / ( 1 - self.beta2 ** self.t)).astype(np.float32)
            
            layer.bias -= (self.learning_rate * m_corrected_bias / (np.sqrt(v_corrected_bias) + self.eps)).astype(np.float32) 

    def zero_grad(self, layers):
        """
        Обнуление градиентов
        """
        for layer in layers:
            if hasattr(layer, 'grad_weight'):
                layer.grad_weight = None
            if hasattr(layer, 'grad_bias'):
                layer.grad_bias = None
            if hasattr(layer, 'grad_gamma'):
                layer.grad_gamma = None
            if hasattr(layer, 'grad_beta'):
                layer.grad_beta = None


In [18]:
# Тест Adam (запустите после реализации Adam)
#print("⚠️ Реализуйте Adam класс выше, затем раскомментируйте этот код для тестирования")

# # Создание тестового слоя
layer = Linear(3, 2)
adam = Adam(learning_rate=0.01)

# # Создание фиктивных градиентов
layer.grad_weight = np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]], dtype=np.float32)
layer.grad_bias = np.array([0.1, 0.2], dtype=np.float32)

# # Сохранение начальных весов
initial_weight = layer.weight.copy()
initial_bias = layer.bias.copy()

print(f"Initial weight: \n{initial_weight}")
print(f"Initial bias: {initial_bias}")
print(f"Weight gradient: \n{layer.grad_weight}")
print(f"Bias gradient: {layer.grad_bias}")

# # Проверим инициализацию Adam
assert len(adam.m) == 0, "Моменты должны быть пустыми при инициализации"
assert len(adam.v) == 0, "Моменты должны быть пустыми при инициализации"
assert adam.t == 0, "Time step должен быть равен 0"

# # Выполним один шаг оптимизации
adam.update(layer, "test_layer")

print(f"\nAfter 1 step:")
print(f"Updated weight: \n{layer.weight}")
print(f"Updated bias: {layer.bias}")
print(f"Time step: {adam.t}")

# # Проверим, что веса изменились
assert not np.allclose(layer.weight, initial_weight), "Веса должны измениться после обновления"
assert not np.allclose(layer.bias, initial_bias), "Bias должен измениться после обновления"

# # Проверим, что моменты инициализированы
assert "test_layer_weight" in adam.m, "Момент для весов должен быть создан"
assert "test_layer_bias" in adam.m, "Момент для bias должен быть создан"
assert "test_layer_weight" in adam.v, "Момент для весов должен быть создан"
assert "test_layer_bias" in adam.v, "Момент для bias должен быть создан"

# # Проверим формы моментов
assert adam.m["test_layer_weight"].shape == layer.weight.shape, "Неправильная форма момента весов"
assert adam.m["test_layer_bias"].shape == layer.bias.shape, "Неправильная форма момента bias"

# # Тест zero_grad
adam.zero_grad([layer])
assert layer.grad_weight is None, "Градиенты весов должны быть обнулены"
assert layer.grad_bias is None, "Градиенты bias должны быть обнулены"

print("✅ Adam тест пройден успешно!")


Initial weight: 
[[0.32108837 0.34717318]
 [0.4521888  1.1152909 ]
 [0.02874159 0.9214675 ]]
Initial bias: [0. 0.]
Weight gradient: 
[[0.1 0.2]
 [0.3 0.4]
 [0.5 0.6]]
Bias gradient: [0.1 0.2]

After 1 step:
Updated weight: 
[[0.31108838 0.3371732 ]
 [0.4421888  1.1052909 ]
 [0.01874159 0.9114675 ]]
Updated bias: [-0.01 -0.01]
Time step: 1
✅ Adam тест пройден успешно!


In [19]:
### Функции потерь

class CrossEntropyLoss:
    def __init__(self):
        self.predictions = None
        self.targets = None
    
    def forward(self, predictions, targets):

        self.predictions = predictions
        self.targets = targets
        
        # TODO: Примените softmax к предсказаниям
        self.softmax_pred = softmax(predictions)
        
        # TODO: Вычислите cross-entropy loss
        clipped_softmax = np.clip(self.softmax_pred, 1e-4, 1 + 1e-4)
        loss = - np.mean(np.log(clipped_softmax[np.arange(predictions.shape[0]), targets]))
        
        return loss
    
    def backward(self):
        """
        Вычисление градиента Cross-Entropy Loss
        
        Returns:
            градиент по предсказаниям
        """
        # TODO: Вычислите градиент
        one_hot_targets = one_hot_encode(self.targets, self.predictions.shape[1])
        
        grad = (self.softmax_pred - one_hot_targets) / self.predictions.shape[0]
        
        return grad


class MSELoss:
    def __init__(self):
        self.predictions = None
        self.targets = None
    
    def forward(self, predictions, targets):
        """
        Вычисление Mean Squared Error
        
        Args:
            predictions: предсказания модели
            targets: истинные значения
        
        Returns:
            значение функции потерь
        """
        self.predictions = predictions
        self.targets = targets
        
        # TODO: Вычислите MSE
        loss = np.mean(np.square(self.predictions - self.targets))
        
        return loss
    
    def backward(self):
        """
        Вычисление градиента MSE
        
        Returns:
            градиент по предсказаниям
        """
        # TODO: Вычислите градиент MSE
        grad = 2 * (self.predictions - self.targets) / self.predictions.shape[0]
        
        return grad


def softmax(x):
    """
    Устойчивая реализация softmax
    """
    # TODO: Реализуйте softmax функцию
    x_shifted = x - np.max(x, axis=-1, keepdims=True)
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)


def one_hot_encode(labels, num_classes):
    """
    Преобразование меток в one-hot кодировку
    """
    # TODO: Создайте one-hot кодировку
    one_hot = np.zeros((len(labels), num_classes))
    one_hot[np.arange(len(labels)), labels] = 1
    return one_hot

In [20]:
# Тест функций потерь (запустите после реализации Loss функций)
#print("⚠️ Реализуйте функции потерь выше, затем раскомментируйте этот код для тестирования")

# # Тест CrossEntropyLoss
print("🔥 Тестирование CrossEntropyLoss...")
ce_loss = CrossEntropyLoss()

# # Тестовые данные
predictions = np.array([[2.0, 1.0, 0.1], [1.0, 3.0, 0.2]], dtype=np.float32)
targets = np.array([0, 1], dtype=np.int32)

print(f"Predictions: \n{predictions}")
print(f"Targets: {targets}")

# # Forward pass
loss_value = ce_loss.forward(predictions, targets)
print(f"CrossEntropy Loss: {loss_value:.4f}")

# # Проверим, что loss положительный
assert loss_value > 0, "CrossEntropy loss должен быть положительным"

# # Backward pass
grad = ce_loss.backward()
print(f"Gradient shape: {grad.shape}")
print(f"Gradient: \n{grad}")

# # Проверим форму градиента
assert grad.shape == predictions.shape, "Неправильная форма градиента CrossEntropy"

# # Проверим, что сумма градиентов по классам равна 0 (свойство softmax)
assert np.allclose(grad.sum(axis=1), 0, atol=1e-6), "Сумма градиентов по классам должна быть 0"

print("✅ CrossEntropyLoss тест пройден!")

# # Тест MSELoss
print("\n📊 Тестирование MSELoss...")
mse_loss = MSELoss()

# # Тестовые данные для регрессии
predictions_reg = np.array([[1.0, 2.0], [3.0, 4.0]], dtype=np.float32)
targets_reg = np.array([[1.5, 2.5], [2.5, 3.5]], dtype=np.float32)

print(f"Predictions: \n{predictions_reg}")
print(f"Targets: \n{targets_reg}")

# # Forward pass
mse_value = mse_loss.forward(predictions_reg, targets_reg)
print(f"MSE Loss: {mse_value:.4f}")

# # Проверим, что loss положительный
assert mse_value >= 0, "MSE loss должен быть неотрицательным"

# # Backward pass
grad_mse = mse_loss.backward()
print(f"MSE Gradient shape: {grad_mse.shape}")
print(f"MSE Gradient: \n{grad_mse}")

# # Проверим форму градиента
assert grad_mse.shape == predictions_reg.shape, "Неправильная форма градиента MSE"

print("✅ MSELoss тест пройден!")

# # Тест softmax функции
print("\n🎯 Тестирование Softmax...")
x_softmax = np.array([[1.0, 2.0, 3.0], [1.0, 1.0, 1.0]], dtype=np.float32)
softmax_output = softmax(x_softmax)

print(f"Input: \n{x_softmax}")
print(f"Softmax output: \n{softmax_output}")

# # Проверим, что сумма вероятностей равна 1
assert np.allclose(softmax_output.sum(axis=1), 1.0), "Сумма softmax должна быть равна 1"

# # Проверим, что все значения положительные
assert np.all(softmax_output > 0), "Все значения softmax должны быть положительными"
assert np.all(softmax_output < 1), "Все значения softmax должны быть меньше 1"

print("✅ Softmax тест пройден!")

# # Тест one-hot encoding
print("\n🏷️ Тестирование One-hot encoding...")
labels = np.array([0, 2, 1, 0])
one_hot = one_hot_encode(labels, num_classes=3)

print(f"Labels: {labels}")
print(f"One-hot: \n{one_hot}")

# # Проверим форму
assert one_hot.shape == (4, 3), "Неправильная форма one-hot кодировки"

# # Проверим, что каждая строка содержит ровно одну единицу
assert np.all(one_hot.sum(axis=1) == 1), "Каждая строка должна содержать ровно одну единицу"

print("✅ One-hot encoding тест пройден!")

print("\n🎉 Все тесты функций потерь пройдены успешно!")


🔥 Тестирование CrossEntropyLoss...
Predictions: 
[[2.  1.  0.1]
 [1.  3.  0.2]]
Targets: [0 1]
CrossEntropy Loss: 0.2981
Gradient shape: (2, 3)
Gradient: 
[[-0.17049941  0.1212165   0.04928295]
 [ 0.05657142 -0.08199063  0.02541918]]
✅ CrossEntropyLoss тест пройден!

📊 Тестирование MSELoss...
Predictions: 
[[1. 2.]
 [3. 4.]]
Targets: 
[[1.5 2.5]
 [2.5 3.5]]
MSE Loss: 0.2500
MSE Gradient shape: (2, 2)
MSE Gradient: 
[[-0.5 -0.5]
 [ 0.5  0.5]]
✅ MSELoss тест пройден!

🎯 Тестирование Softmax...
Input: 
[[1. 2. 3.]
 [1. 1. 1.]]
Softmax output: 
[[0.09003057 0.24472848 0.66524094]
 [0.33333334 0.33333334 0.33333334]]
✅ Softmax тест пройден!

🏷️ Тестирование One-hot encoding...
Labels: [0 2 1 0]
One-hot: 
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
✅ One-hot encoding тест пройден!

🎉 Все тесты функций потерь пройдены успешно!


In [21]:
class NeuralNetwork:
    def __init__(self):
        # TODO: Создайте архитектуру нейронной сети
        self.model = Sequential(Linear(784, 512),
                                BatchNorm(512),
                                ReLU(),
                                Dropout(dropout_rate=0.5),
                                Linear(512, 256),
                                BatchNorm(256),
                                ReLU(),
                                Dropout(dropout_rate=0.3),
                                Linear(256, 128),
                                BatchNorm(128),
                                ReLU(),
                                Dropout(dropout_rate=0.2),
                                Linear(128, 10)

        )
    
    def forward(self, x):
        return self.model.forward(x)
    
    def backward(self, grad_output):
        return self.model.backward(grad_output)
    
    def train(self):
        self.model.train()
    
    def eval(self):
        self.model.eval()
    
    def get_trainable_layers(self):
        """
        Получение всех слоев с обучаемыми параметрами
        """
        trainable_layers = []
        for layer in self.model.layers:
            if hasattr(layer, 'update_weights'):
                trainable_layers.append(layer)
        return trainable_layers

In [22]:
import torch
torch.cuda.is_available()

False

In [23]:
device = torch.device('cuda') 

In [25]:
model = NeuralNetwork()  ### Не понял как на device перенести
criterion = CrossEntropyLoss()
optimizer = Adam(learning_rate=0.001)

import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X = dataset.drop('label', axis=1).values.astype(np.float32) / 255.0
y = dataset['label'].values

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер валидационной выборки: {X_val.shape}")

X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_train = y_train.astype(np.int64)
y_val = y_val.astype(np.int64)

#/kaggle/input/digit-recognizer

Размер обучающей выборки: (33600, 784)
Размер валидационной выборки: (8400, 784)


In [26]:
def create_batches(X, y, batch_size, shuffle=True):
    """Создание батчей для обучения"""
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    
    if shuffle:
        np.random.shuffle(indices)
    
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        batch_indices = indices[start_idx:end_idx]
        yield X[batch_indices], y[batch_indices]

def calculate_accuracy(predictions, targets):
    """Вычисление точности"""
    predicted_classes = np.argmax(predictions, axis=1)
    return np.mean(predicted_classes == targets)

In [ ]:
def train_model(model, optimizer, criterion, X_train, y_train, X_val, y_val, 
                num_epochs=30, batch_size=128, print_every=100, max_grad_norm=1.0):
    """
    Цикл тренировки нейросети
    """
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        # Режим обучения
        model.train()
        epoch_train_loss = 0.0
        epoch_train_acc = 0.0
        num_batches = 0

        indices = np.random.permutation(len(X_train))
        X_train_shuffled = X_train[indices]
        y_train_shuffled = y_train[indices]
        
        # Обучение на батчах
        for batch_X, batch_y in create_batches(X_train_shuffled, y_train_shuffled, batch_size):
            # Forward pass
            predictions = model.forward(batch_X)
            loss = criterion.forward(predictions, batch_y)
            
            # Backward pass
            grad_output = criterion.backward()
            model.backward(grad_output)
            
            # Обновление параметров
            total_norm = 0
            trainable_layers = model.get_trainable_layers()
            for layer in trainable_layers:
                if hasattr(layer, 'grad_weight') and layer.grad_weight is not None:
                    total_norm += np.sum(layer.grad_weight ** 2)
                if hasattr(layer, 'grad_bias') and layer.grad_bias is not None:
                    total_norm += np.sum(layer.grad_bias ** 2)
            total_norm = np.sqrt(total_norm)

            if total_norm > max_grad_norm:
                clip_coef = max_grad_norm / (total_norm + 1e-6)
                for layer in trainable_layers:
                    if hasattr(layer, 'grad_weight') and layer.grad_weight is not None:
                        layer.grad_weight *= clip_coef
                    if hasattr(layer, 'grad_bias') and layer.grad_bias is not None:
                        layer.grad_bias *= clip_coef

            for i, layer in enumerate(trainable_layers):
                optimizer.update(layer, f"layer_{i}")
            
            # Обнуление градиентов
            optimizer.zero_grad(model.model.layers)
            
            
            # Статистика
            epoch_train_loss += loss
            epoch_train_acc += calculate_accuracy(predictions, batch_y)
            num_batches += 1
            
            if num_batches % print_every == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{num_batches}], "
                      f"Loss: {loss:.4f}, Accuracy: {calculate_accuracy(predictions, batch_y):.4f}")
        
        # Средние значения за эпоху
        epoch_train_loss /= num_batches
        epoch_train_acc /= num_batches
        
        # Валидация
        model.eval()
        val_predictions = model.forward(X_val)
        val_loss = criterion.forward(val_predictions, y_val)
        val_acc = calculate_accuracy(val_predictions, y_val)
        
        # Сохранение статистики
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    return train_losses, train_accuracies, val_losses, val_accuracies

# Запуск обучения
print("Начинаем обучение...")
train_losses, train_accs, val_losses, val_accs = train_model(
    model, optimizer, criterion, X_train, y_train, X_val, y_val,
    num_epochs=40, batch_size=256, print_every=50
)

print("Обучение завершено!")

In [ ]:
import pandas as pd
import numpy as np

test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

X_test = test_data.values.astype(np.float32) / 255.0  # Нормализация к [0, 1]

In [ ]:
def predict_on_test_data(model, X_test, batch_size=256):

    model.eval()
    
    all_predictions = []
    num_samples = X_test.shape[0]

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_X = X_test[start_idx:end_idx]

        batch_predictions = model.forward(batch_X)
        
        batch_probs = softmax(batch_predictions)
        
        batch_classes = np.argmax(batch_probs, axis=1)
        
        all_predictions.extend(batch_classes)
    
    return np.array(all_predictions)

In [ ]:
def create_submission_file(predictions, filename='submission.csv'):
    submission = pd.DataFrame({
        'ImageId': range(1, len(predictions) + 1),  # ImageId начинается с 1
        'Label': predictions
    })

    submission.to_csv(filename, index=False)
    return submission

submission_df = create_submission_file(predictions, 'mnist_submission.csv')